# CNN + LSTM Model for Key Press Detection

This notebook trains a CNN+LSTM model to detect key press events from video sequences. The model combines:
- **CNN**: Extracts spatial features from key region images
- **LSTM**: Models temporal patterns for key press detection

## Dataset Structure
The training data comes from the video labeler application with the following format:
- Image sequences (64x64x3) of key regions
- Binary labels (0: not pressed, 1: pressed)
- Temporal ordering for sequence modeling

## Model Architecture
1. **Feature Extraction**: CNN processes individual frames
2. **Temporal Modeling**: LSTM learns temporal patterns
3. **Classification**: Dense layer outputs key press probability

In [ ]:
# Import Required Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
import json
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.experimental.list_physical_devices('GPU'))
print("Libraries imported successfully!")

In [ ]:
# Load and Preprocess Training Data
def load_training_data(data_dir="labeled_data"):
    """Load training data from JSON files exported by video labeler."""
    
    # Find all training data files
    json_files = glob.glob(os.path.join(data_dir, "training_data_*.json"))
    
    if not json_files:
        raise FileNotFoundError(f"No training data found in {data_dir}")
    
    all_images = []
    all_labels = []
    all_frame_indices = []
    
    print(f"Found {len(json_files)} training data files:")
    
    for json_file in json_files:
        print(f"Loading: {json_file}")
        
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        # Extract sequence data
        sequence_data = data['sequence_data']
        
        for item in sequence_data:
            # Convert image list back to numpy array
            image = np.array(item['image'], dtype=np.float32)
            
            # Ensure image is in correct format (64, 64, 3)
            if image.shape != (64, 64, 3):
                print(f"Warning: Invalid image shape {image.shape}, skipping...")
                continue
                
            all_images.append(image)
            all_labels.append(item['label'])
            all_frame_indices.append(item['frame_idx'])
        
        print(f"  - Loaded {len(sequence_data)} samples")
    
    # Convert to numpy arrays
    X = np.array(all_images)
    y = np.array(all_labels)
    frame_indices = np.array(all_frame_indices)
    
    print(f"\nTotal dataset:")
    print(f"  - Images shape: {X.shape}")
    print(f"  - Labels shape: {y.shape}")
    print(f"  - Label distribution: {np.bincount(y)}")
    
    return X, y, frame_indices

# Load the data
try:
    X, y, frame_indices = load_training_data()
    print("Data loaded successfully!")
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please make sure you have exported training data from the video labeler.")

In [ ]:
# Data Exploration and Visualization
def visualize_dataset(X, y, frame_indices, num_samples=8):
    """Visualize sample images and analyze dataset."""
    
    if len(X) == 0:
        print("No data to visualize")
        return
    
    # Plot sample images
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle('Sample Key Region Images', fontsize=16)
    
    # Show samples from each class
    pressed_indices = np.where(y == 1)[0]
    not_pressed_indices = np.where(y == 0)[0]
    
    for i in range(4):
        # Not pressed samples
        if i < len(not_pressed_indices):
            idx = not_pressed_indices[i]
            axes[0, i].imshow(X[idx])
            axes[0, i].set_title(f'Not Pressed (Frame {frame_indices[idx]})')
            axes[0, i].axis('off')
        
        # Pressed samples
        if i < len(pressed_indices):
            idx = pressed_indices[i]
            axes[1, i].imshow(X[idx])
            axes[1, i].set_title(f'Pressed (Frame {frame_indices[idx]})')
            axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Label distribution
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    labels_count = np.bincount(y)
    plt.bar(['Not Pressed', 'Pressed'], labels_count, 
            color=['lightcoral', 'lightgreen'])
    plt.title('Label Distribution')
    plt.ylabel('Count')
    
    # Add percentage labels
    total = len(y)
    for i, count in enumerate(labels_count):
        plt.text(i, count + total*0.01, f'{count}\n({count/total:.1%})', 
                ha='center', va='bottom')
    
    plt.subplot(1, 2, 2)
    plt.hist(frame_indices, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Frame Index Distribution')
    plt.xlabel('Frame Index')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
    
    # Dataset statistics
    print(f"Dataset Statistics:")
    print(f"  - Total samples: {len(X)}")
    print(f"  - Image shape: {X.shape[1:]}")
    print(f"  - Image value range: [{X.min():.3f}, {X.max():.3f}]")
    print(f"  - Not pressed samples: {labels_count[0]} ({labels_count[0]/total:.1%})")
    print(f"  - Pressed samples: {labels_count[1]} ({labels_count[1]/total:.1%})")

# Visualize the dataset
if 'X' in locals() and len(X) > 0:
    visualize_dataset(X, y, frame_indices)
else:
    print("No data available for visualization")

In [ ]:
# Data Augmentation and Preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_data_augmentation():
    """Create data augmentation pipeline."""
    
    # Define augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=10,          # Random rotation
        width_shift_range=0.1,      # Random horizontal shift
        height_shift_range=0.1,     # Random vertical shift
        brightness_range=[0.8, 1.2], # Random brightness
        zoom_range=0.1,             # Random zoom
        horizontal_flip=False,       # No horizontal flip (text orientation)
        fill_mode='nearest',        # Fill mode for transformations
        rescale=None               # Don't rescale (already normalized)
    )
    
    return datagen

def augment_dataset(X, y, augment_factor=2):
    """Augment dataset to increase sample size."""
    
    if len(X) == 0:
        return X, y
    
    print(f"Original dataset size: {len(X)}")
    
    # Separate classes
    pressed_indices = np.where(y == 1)[0]
    not_pressed_indices = np.where(y == 0)[0]
    
    # Balance classes by augmenting minority class
    if len(pressed_indices) < len(not_pressed_indices):
        minority_class = 1
        minority_indices = pressed_indices
        majority_indices = not_pressed_indices
    else:
        minority_class = 0
        minority_indices = not_pressed_indices
        majority_indices = pressed_indices
    
    print(f"Minority class: {minority_class} ({len(minority_indices)} samples)")
    print(f"Majority class: {1-minority_class} ({len(majority_indices)} samples)")
    
    # Create augmentation generator
    datagen = create_data_augmentation()
    
    # Augment minority class
    X_minority = X[minority_indices]
    y_minority = y[minority_indices]
    
    # Generate augmented samples
    augmented_X = []
    augmented_y = []
    
    target_size = len(majority_indices)
    samples_needed = target_size - len(minority_indices)
    
    if samples_needed > 0:
        samples_per_original = samples_needed // len(minority_indices) + 1
        
        for i, (img, label) in enumerate(zip(X_minority, y_minority)):
            # Add original sample
            augmented_X.append(img)
            augmented_y.append(label)
            
            # Generate augmented samples
            img_batch = np.expand_dims(img, axis=0)
            aug_iter = datagen.flow(img_batch, batch_size=1, shuffle=False)
            
            for _ in range(samples_per_original):
                if len(augmented_X) >= target_size:
                    break
                aug_img = next(aug_iter)[0]
                augmented_X.append(aug_img)
                augmented_y.append(label)
    
    # Combine original majority class with augmented minority class
    X_balanced = np.concatenate([X[majority_indices], np.array(augmented_X[:target_size])])
    y_balanced = np.concatenate([y[majority_indices], np.array(augmented_y[:target_size])])
    
    # Shuffle the balanced dataset
    shuffle_indices = np.random.permutation(len(X_balanced))
    X_balanced = X_balanced[shuffle_indices]
    y_balanced = y_balanced[shuffle_indices]
    
    print(f"Balanced dataset size: {len(X_balanced)}")
    print(f"New class distribution: {np.bincount(y_balanced)}")
    
    return X_balanced, y_balanced

def preprocess_data(X, y, balance_classes=True):
    """Preprocess data for training."""
    
    if len(X) == 0:
        return X, y
    
    # Ensure data is in correct format
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int32)
    
    # Clip values to [0, 1] range (should already be normalized)
    X = np.clip(X, 0.0, 1.0)
    
    # Balance classes if requested
    if balance_classes:
        X, y = augment_dataset(X, y)
    
    # Convert labels to categorical
    y_categorical = keras.utils.to_categorical(y, num_classes=2)
    
    print(f"Preprocessed dataset:")
    print(f"  - X shape: {X.shape}")
    print(f"  - y shape: {y_categorical.shape}")
    print(f"  - X range: [{X.min():.3f}, {X.max():.3f}]")
    
    return X, y_categorical

# Preprocess the data
if 'X' in locals() and len(X) > 0:
    X_processed, y_processed = preprocess_data(X, y)
    print("Data preprocessing completed!")
else:
    print("No data available for preprocessing")

In [ ]:
# Create CNN Feature Extractor
def create_cnn_feature_extractor(input_shape=(64, 64, 3)):
    """Create CNN model for feature extraction from key region images."""
    
    model = keras.Sequential([
        # Input layer
        layers.Input(shape=input_shape),
        
        # First convolutional block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Second convolutional block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Third convolutional block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Fourth convolutional block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Global average pooling for feature extraction
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5)
    ])
    
    return model

# Create and display CNN feature extractor
cnn_feature_extractor = create_cnn_feature_extractor()
print("CNN Feature Extractor Architecture:")
cnn_feature_extractor.summary()

# Test the feature extractor with sample input
if 'X_processed' in locals() and len(X_processed) > 0:
    sample_input = X_processed[:1]  # Take one sample
    features = cnn_feature_extractor(sample_input)
    print(f"\nFeature extraction test:")
    print(f"Input shape: {sample_input.shape}")
    print(f"Output features shape: {features.shape}")
    print(f"Feature vector size: {features.shape[1]}")
else:
    print("\nNo processed data available for testing")

In [ ]:
# Build CNN+LSTM Model Architecture
def create_cnn_lstm_model(sequence_length=10, input_shape=(64, 64, 3), num_classes=2):
    """Create CNN+LSTM model for temporal key press detection."""
    
    # Input for sequence of images
    sequence_input = layers.Input(shape=(sequence_length,) + input_shape)
    
    # CNN feature extractor (TimeDistributed to process each frame)
    cnn_features = layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))(sequence_input)
    cnn_features = layers.TimeDistributed(layers.BatchNormalization())(cnn_features)
    cnn_features = layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.Dropout(0.25))(cnn_features)
    
    cnn_features = layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.BatchNormalization())(cnn_features)
    cnn_features = layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.Dropout(0.25))(cnn_features)
    
    cnn_features = layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.BatchNormalization())(cnn_features)
    cnn_features = layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.Dropout(0.25))(cnn_features)
    
    # Global average pooling for each frame
    cnn_features = layers.TimeDistributed(layers.GlobalAveragePooling2D())(cnn_features)
    cnn_features = layers.TimeDistributed(layers.Dense(256, activation='relu'))(cnn_features)
    cnn_features = layers.TimeDistributed(layers.BatchNormalization())(cnn_features)
    cnn_features = layers.TimeDistributed(layers.Dropout(0.5))(cnn_features)
    
    # LSTM layers for temporal modeling
    lstm_features = layers.LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(cnn_features)
    lstm_features = layers.LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.3)(lstm_features)
    
    # Final classification layers
    dense_features = layers.Dense(128, activation='relu')(lstm_features)
    dense_features = layers.BatchNormalization()(dense_features)
    dense_features = layers.Dropout(0.5)(dense_features)
    
    dense_features = layers.Dense(64, activation='relu')(dense_features)
    dense_features = layers.BatchNormalization()(dense_features)
    dense_features = layers.Dropout(0.5)(dense_features)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(dense_features)
    
    # Create model
    model = keras.Model(inputs=sequence_input, outputs=outputs)
    
    return model

# Alternative: Simpler CNN+LSTM model for single frame prediction
def create_simple_cnn_lstm_model(input_shape=(64, 64, 3), num_classes=2):
    """Create simpler CNN+LSTM model for single frame prediction."""
    
    # Input
    inputs = layers.Input(shape=input_shape)
    
    # CNN feature extraction
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    
    # Dense layers
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Output
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create the model (start with simpler version)
print("Creating CNN+LSTM Model...")
model = create_simple_cnn_lstm_model()

print("\nModel Architecture:")
model.summary()

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

print("\nModel compiled successfully!")
print(f"Total parameters: {model.count_params():,}")

In [ ]:
# Prepare Sequential Data for Training
def create_sequences(X, y, sequence_length=5):
    """Create sequences for temporal modeling."""
    
    if len(X) < sequence_length:
        print(f"Warning: Not enough data for sequence length {sequence_length}")
        return X, y
    
    sequences_X = []
    sequences_y = []
    
    # Create sliding window sequences
    for i in range(len(X) - sequence_length + 1):
        seq_x = X[i:i + sequence_length]
        seq_y = y[i + sequence_length - 1]  # Use last frame's label
        
        sequences_X.append(seq_x)
        sequences_y.append(seq_y)
    
    return np.array(sequences_X), np.array(sequences_y)

def prepare_training_data(X, y, test_size=0.2, validation_size=0.2, use_sequences=False):
    """Prepare data for training with train/validation/test splits."""
    
    if len(X) == 0:
        print("No data available for training")
        return None
    
    print(f"Preparing training data...")
    print(f"Original dataset size: {len(X)}")
    
    # Create sequences if requested
    if use_sequences:
        X, y = create_sequences(X, y)
        print(f"Sequence dataset size: {len(X)}")
    
    # Split data
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y.argmax(axis=1)
    )
    
    # Further split temp into train and validation
    val_size_adjusted = validation_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_size_adjusted, random_state=42, 
        stratify=y_temp.argmax(axis=1)
    )
    
    print(f"Data splits:")
    print(f"  - Training: {len(X_train)} samples")
    print(f"  - Validation: {len(X_val)} samples")
    print(f"  - Test: {len(X_test)} samples")
    
    # Print class distribution for each split
    for name, labels in [("Training", y_train), ("Validation", y_val), ("Test", y_test)]:
        class_counts = labels.argmax(axis=1)
        unique, counts = np.unique(class_counts, return_counts=True)
        print(f"  - {name} class distribution: {dict(zip(unique, counts))}")
    
    return {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test
    }

def create_data_generators(data_dict, batch_size=32, augment_training=True):
    """Create data generators for efficient training."""
    
    if augment_training:
        # Training data generator with augmentation
        train_datagen = ImageDataGenerator(
            rotation_range=5,
            width_shift_range=0.05,
            height_shift_range=0.05,
            brightness_range=[0.9, 1.1],
            zoom_range=0.05,
            horizontal_flip=False,
            fill_mode='nearest'
        )
        
        train_generator = train_datagen.flow(
            data_dict['X_train'], data_dict['y_train'],
            batch_size=batch_size, shuffle=True
        )
    else:
        # Simple batch generator without augmentation
        train_generator = tf.data.Dataset.from_tensor_slices(
            (data_dict['X_train'], data_dict['y_train'])
        ).batch(batch_size).shuffle(1000).prefetch(tf.data.AUTOTUNE)
    
    # Validation generator (no augmentation)
    val_generator = tf.data.Dataset.from_tensor_slices(
        (data_dict['X_val'], data_dict['y_val'])
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return train_generator, val_generator

# Prepare the data
if 'X_processed' in locals() and len(X_processed) > 0:
    # Prepare training data
    data_splits = prepare_training_data(X_processed, y_processed, use_sequences=False)
    
    if data_splits:
        # Create data generators
        train_gen, val_gen = create_data_generators(data_splits, batch_size=32)
        
        print("Data preparation completed!")
        print(f"Training steps per epoch: {len(data_splits['X_train']) // 32}")
        print(f"Validation steps per epoch: {len(data_splits['X_val']) // 32}")
    else:
        print("Failed to prepare data splits")
else:
    print("No processed data available for training preparation")

In [ ]:
# Train the Model
def create_callbacks(model_name="keypress_model"):
    """Create callbacks for training."""
    
    # Create model directory
    model_dir = f"models/{model_name}"
    os.makedirs(model_dir, exist_ok=True)
    
    callbacks = [
        # Model checkpoint - save best model
        keras.callbacks.ModelCheckpoint(
            filepath=f"{model_dir}/best_model.h5",
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            verbose=1
        ),
        
        # Early stopping
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Learning rate reduction
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-7,
            verbose=1
        ),
        
        # CSV logger
        keras.callbacks.CSVLogger(
            f"{model_dir}/training_log.csv",
            append=True
        ),
        
        # TensorBoard
        keras.callbacks.TensorBoard(
            log_dir=f"{model_dir}/tensorboard_logs",
            histogram_freq=1,
            write_graph=True,
            write_images=True
        )
    ]
    
    return callbacks

def train_model(model, train_data, val_data, epochs=100, model_name="keypress_model"):
    """Train the model with callbacks."""
    
    print(f"Starting training for {epochs} epochs...")
    print(f"Model will be saved to: models/{model_name}/")
    
    # Create callbacks
    callbacks = create_callbacks(model_name)
    
    # Train the model
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    return history

def plot_training_history(history):
    """Plot training history."""
    
    if not history:
        print("No training history available")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot training & validation accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Plot training & validation loss
    axes[0, 1].plot(history.history['loss'], label='Training Loss')
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Plot precision
    if 'precision' in history.history:
        axes[1, 0].plot(history.history['precision'], label='Training Precision')
        axes[1, 0].plot(history.history['val_precision'], label='Validation Precision')
        axes[1, 0].set_title('Model Precision')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Precision')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    # Plot recall
    if 'recall' in history.history:
        axes[1, 1].plot(history.history['recall'], label='Training Recall')
        axes[1, 1].plot(history.history['val_recall'], label='Validation Recall')
        axes[1, 1].set_title('Model Recall')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Recall')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

# Train the model
if 'model' in locals() and 'data_splits' in locals():
    print("Starting model training...")
    
    # Create model directory
    os.makedirs("models", exist_ok=True)
    
    # Train the model
    history = train_model(
        model=model,
        train_data=train_gen,
        val_data=val_gen,
        epochs=50,  # Start with 50 epochs
        model_name="cnn_lstm_keypress"
    )
    
    # Plot training history
    plot_training_history(history)
    
    print("Training completed!")
    
else:
    print("Model or data not available for training")
    print("Please ensure you have:")
    print("1. Loaded and preprocessed training data")
    print("2. Created the model")
    print("3. Prepared data splits")

In [ ]:
# Evaluate Model Performance
def evaluate_model(model, X_test, y_test):
    """Evaluate model on test data."""
    
    print("Evaluating model on test data...")
    
    # Make predictions
    predictions = model.predict(X_test, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    y_true = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"Test Results:")
    print(f"  - Accuracy: {accuracy:.4f}")
    print(f"  - Precision: {precision:.4f}")
    print(f"  - Recall: {recall:.4f}")
    print(f"  - F1-Score: {f1:.4f}")
    
    return y_pred, y_true, predictions

def plot_confusion_matrix(y_true, y_pred, class_names=['Not Pressed', 'Pressed']):
    """Plot confusion matrix."""
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    # Print detailed metrics
    print("\nDetailed Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

def plot_prediction_confidence(predictions, y_true, num_samples=100):
    """Plot prediction confidence distribution."""
    
    # Select random samples
    indices = np.random.choice(len(predictions), min(num_samples, len(predictions)), replace=False)
    
    confidence_scores = np.max(predictions[indices], axis=1)
    true_labels = y_true[indices]
    
    # Separate by correct/incorrect predictions
    pred_labels = np.argmax(predictions[indices], axis=1)
    correct_mask = (pred_labels == true_labels)
    
    plt.figure(figsize=(12, 4))
    
    # Plot confidence for correct predictions
    plt.subplot(1, 2, 1)
    plt.hist(confidence_scores[correct_mask], bins=20, alpha=0.7, 
             color='green', label='Correct Predictions')
    plt.hist(confidence_scores[~correct_mask], bins=20, alpha=0.7, 
             color='red', label='Incorrect Predictions')
    plt.xlabel('Prediction Confidence')
    plt.ylabel('Frequency')
    plt.title('Prediction Confidence Distribution')
    plt.legend()
    
    # Plot confidence by class
    plt.subplot(1, 2, 2)
    for class_idx in range(2):
        class_mask = (true_labels == class_idx)
        plt.hist(confidence_scores[class_mask], bins=20, alpha=0.7, 
                 label=f'Class {class_idx}')
    plt.xlabel('Prediction Confidence')
    plt.ylabel('Frequency')
    plt.title('Confidence by True Class')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def analyze_errors(X_test, y_true, y_pred, predictions, num_errors=8):
    """Analyze prediction errors."""
    
    # Find incorrect predictions
    incorrect_indices = np.where(y_true != y_pred)[0]
    
    if len(incorrect_indices) == 0:
        print("No prediction errors found!")
        return
    
    print(f"Found {len(incorrect_indices)} prediction errors out of {len(y_true)} samples")
    print(f"Error rate: {len(incorrect_indices)/len(y_true):.2%}")
    
    # Show some error cases
    num_to_show = min(num_errors, len(incorrect_indices))
    error_indices = np.random.choice(incorrect_indices, num_to_show, replace=False)
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle('Prediction Errors Analysis', fontsize=16)
    
    for i, idx in enumerate(error_indices):
        row = i // 4
        col = i % 4
        
        axes[row, col].imshow(X_test[idx])
        
        true_label = "Pressed" if y_true[idx] == 1 else "Not Pressed"
        pred_label = "Pressed" if y_pred[idx] == 1 else "Not Pressed"
        confidence = predictions[idx][y_pred[idx]]
        
        axes[row, col].set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.3f}')
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Evaluate the model
if 'model' in locals() and 'data_splits' in locals():
    # Load best model if available
    try:
        best_model = keras.models.load_model("models/cnn_lstm_keypress/best_model.h5")
        print("Loaded best model from checkpoint")
        model = best_model
    except:
        print("Using current model (no checkpoint found)")
    
    # Evaluate on test data
    y_pred, y_true, predictions = evaluate_model(model, data_splits['X_test'], data_splits['y_test'])
    
    # Plot confusion matrix
    plot_confusion_matrix(y_true, y_pred)
    
    # Plot prediction confidence
    plot_prediction_confidence(predictions, y_true)
    
    # Analyze errors
    analyze_errors(data_splits['X_test'], y_true, y_pred, predictions)
    
else:
    print("Model or data not available for evaluation")
    print("Please ensure you have trained the model first")

In [ ]:
# Real-time Inference Testing
def create_inference_pipeline(model, target_key='t', model_path='best_v2.pt'):
    """Create inference pipeline for real-time testing."""
    
    from ultralytics import YOLO
    
    # Load YOLO model for key detection
    yolo_model = YOLO(model_path)
    
    def predict_keypress(frame, key_bbox=None):
        """Predict keypress from a single frame."""
        
        if key_bbox is None:
            # Detect key in frame
            results = yolo_model(frame)
            
            for result in results:
                if result.boxes is not None:
                    for box in result.boxes:
                        if result.names[int(box.cls)] == target_key:
                            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                            key_bbox = (int(x1), int(y1), int(x2), int(y2))
                            break
            
            if key_bbox is None:
                return None, None  # Key not found
        
        # Extract and preprocess key region
        x1, y1, x2, y2 = key_bbox
        
        # Expand bounding box by 20% (same as video labeler)
        width = x2 - x1
        height = y2 - y1
        expand_x = width * 0.2
        expand_y = height * 0.2
        
        exp_x1 = max(0, int(x1 - expand_x))
        exp_y1 = max(0, int(y1 - expand_y))
        exp_x2 = min(frame.shape[1], int(x2 + expand_x))
        exp_y2 = min(frame.shape[0], int(y2 + expand_y))
        
        # Extract key region
        key_region = frame[exp_y1:exp_y2, exp_x1:exp_x2]
        
        # Resize to model input size
        key_region = cv2.resize(key_region, (64, 64))
        
        # Normalize
        key_region = key_region.astype(np.float32) / 255.0
        
        # Add batch dimension
        key_region = np.expand_dims(key_region, axis=0)
        
        # Make prediction
        prediction = model.predict(key_region, verbose=0)
        
        # Get probability and class
        prob = prediction[0]
        predicted_class = np.argmax(prob)
        confidence = prob[predicted_class]
        
        return predicted_class, confidence
    
    return predict_keypress

def test_on_video(video_path, model, target_key='t', output_path=None):
    """Test model on a video file."""
    
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video: {video_path}")
    print(f"FPS: {fps}, Size: {width}x{height}, Frames: {total_frames}")
    
    # Create inference pipeline
    predict_keypress = create_inference_pipeline(model, target_key)
    
    # Setup video writer if output path is provided
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Process video
    frame_count = 0
    key_bbox = None
    predictions = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Predict keypress
        predicted_class, confidence = predict_keypress(frame, key_bbox)
        
        if predicted_class is not None:
            predictions.append({
                'frame': frame_count,
                'prediction': predicted_class,
                'confidence': confidence
            })
            
            # Draw prediction on frame
            label = "PRESSED" if predicted_class == 1 else "NOT PRESSED"
            color = (0, 255, 0) if predicted_class == 1 else (0, 0, 255)
            
            cv2.putText(frame, f"{label} ({confidence:.3f})", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            
            # Draw key bounding box if found
            if key_bbox:
                x1, y1, x2, y2 = key_bbox
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        # Write frame to output video
        if output_path:
            out.write(frame)
        
        # Display frame (optional)
        cv2.imshow('Keypress Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()
    
    print(f"Processed {frame_count} frames")
    print(f"Made {len(predictions)} predictions")
    
    return predictions

def analyze_video_predictions(predictions):
    """Analyze predictions from video inference."""
    
    if not predictions:
        print("No predictions to analyze")
        return
    
    # Convert to DataFrame for easier analysis
    df = pd.DataFrame(predictions)
    
    # Basic statistics
    print(f"Video Prediction Analysis:")
    print(f"  - Total frames predicted: {len(df)}")
    print(f"  - Pressed frames: {sum(df['prediction'] == 1)}")
    print(f"  - Not pressed frames: {sum(df['prediction'] == 0)}")
    print(f"  - Average confidence: {df['confidence'].mean():.3f}")
    
    # Plot predictions over time
    plt.figure(figsize=(15, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(df['frame'], df['prediction'], 'b-', linewidth=2)
    plt.title('Key Press Predictions Over Time')
    plt.ylabel('Prediction (0=Not Pressed, 1=Pressed)')
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.plot(df['frame'], df['confidence'], 'r-', linewidth=2)
    plt.title('Prediction Confidence Over Time')
    plt.xlabel('Frame Number')
    plt.ylabel('Confidence')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Test the model on video
if 'model' in locals():
    # Example usage (uncomment and modify paths as needed)
    
    # Test on a video file
    # video_path = "path/to/your/test_video.mp4"
    # output_path = "keypress_predictions.mp4"
    
    # predictions = test_on_video(video_path, model, target_key='t', output_path=output_path)
    # analyze_video_predictions(predictions)
    
    print("Real-time inference pipeline created!")
    print("To test on a video:")
    print("1. Uncomment the lines above")
    print("2. Set video_path to your test video")
    print("3. Set output_path for result video")
    print("4. Run the cell")
    
else:
    print("Model not available for inference testing")
    print("Please train the model first")

In [ ]:
# Model Optimization and Fine-tuning
def create_optimized_model(input_shape=(64, 64, 3), num_classes=2):
    """Create optimized CNN model with better architecture."""
    
    inputs = layers.Input(shape=input_shape)
    
    # Initial convolution
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    
    # Residual block 1
    residual = x
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Residual block 2
    residual = layers.Conv2D(64, (1, 1), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Residual block 3
    residual = layers.Conv2D(128, (1, 1), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Attention mechanism
    attention = layers.Conv2D(128, (1, 1), activation='sigmoid')(x)
    x = layers.Multiply()([x, attention])
    
    # Global pooling and dense layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

def fine_tune_model(base_model, X_train, y_train, X_val, y_val, epochs=20):
    """Fine-tune model with different learning rates."""
    
    print("Fine-tuning model...")
    
    # Freeze some layers
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    # Compile with lower learning rate
    base_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Create callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7
        )
    ]
    
    # Fine-tune
    history = base_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    return history

def hyperparameter_tuning(X_train, y_train, X_val, y_val):
    """Perform hyperparameter tuning."""
    
    print("Performing hyperparameter tuning...")
    
    # Define hyperparameter search space
    lr_values = [0.001, 0.0005, 0.0001]
    batch_sizes = [16, 32, 64]
    dropout_rates = [0.3, 0.5, 0.7]
    
    best_accuracy = 0
    best_params = {}
    results = []
    
    for lr in lr_values:
        for batch_size in batch_sizes:
            for dropout in dropout_rates:
                print(f"Testing: LR={lr}, Batch={batch_size}, Dropout={dropout}")
                
                # Create model with current hyperparameters
                model = create_optimized_model()
                
                # Modify dropout rate
                for layer in model.layers:
                    if isinstance(layer, layers.Dropout):
                        layer.rate = dropout
                
                # Compile model
                model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=lr),
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                )
                
                # Train for a few epochs
                history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=batch_size,
                    verbose=0
                )
                
                # Get best validation accuracy
                val_accuracy = max(history.history['val_accuracy'])
                
                results.append({
                    'lr': lr,
                    'batch_size': batch_size,
                    'dropout': dropout,
                    'val_accuracy': val_accuracy
                })
                
                if val_accuracy > best_accuracy:
                    best_accuracy = val_accuracy
                    best_params = {
                        'lr': lr,
                        'batch_size': batch_size,
                        'dropout': dropout
                    }
                
                print(f"  Validation accuracy: {val_accuracy:.4f}")
    
    print(f"\nBest parameters: {best_params}")
    print(f"Best validation accuracy: {best_accuracy:.4f}")
    
    return best_params, results

def ensemble_prediction(models, X_test):
    """Create ensemble predictions from multiple models."""
    
    predictions = []
    
    for model in models:
        pred = model.predict(X_test, verbose=0)
        predictions.append(pred)
    
    # Average predictions
    ensemble_pred = np.mean(predictions, axis=0)
    
    return ensemble_pred

# Model optimization example
if 'data_splits' in locals():
    print("Starting model optimization...")
    
    # Create optimized model
    optimized_model = create_optimized_model()
    
    print("Optimized Model Architecture:")
    optimized_model.summary()
    
    # Compile optimized model
    optimized_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Optional: Perform hyperparameter tuning (this will take a while)
    # best_params, tuning_results = hyperparameter_tuning(
    #     data_splits['X_train'], data_splits['y_train'],
    #     data_splits['X_val'], data_splits['y_val']
    # )
    
    print("Model optimization setup completed!")
    print("Uncomment hyperparameter tuning section to run full optimization")
    
else:
    print("Data not available for model optimization")
    print("Please prepare training data first")

In [ ]:
# Export Model for Production
def export_model(model, model_name="keypress_detector"):
    """Export trained model in multiple formats for production."""
    
    # Create export directory
    export_dir = f"exported_models/{model_name}"
    os.makedirs(export_dir, exist_ok=True)
    
    print(f"Exporting model to {export_dir}/...")
    
    # 1. Save in Keras H5 format
    h5_path = f"{export_dir}/{model_name}.h5"
    model.save(h5_path)
    print(f"✓ Saved H5 model: {h5_path}")
    
    # 2. Save in TensorFlow SavedModel format
    savedmodel_path = f"{export_dir}/{model_name}_savedmodel"
    model.save(savedmodel_path)
    print(f"✓ Saved TensorFlow SavedModel: {savedmodel_path}")
    
    # 3. Save model weights only
    weights_path = f"{export_dir}/{model_name}_weights.h5"
    model.save_weights(weights_path)
    print(f"✓ Saved model weights: {weights_path}")
    
    # 4. Export to TensorFlow Lite for mobile deployment
    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        
        tflite_path = f"{export_dir}/{model_name}.tflite"
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"✓ Saved TensorFlow Lite model: {tflite_path}")
        
        # Get model size
        model_size = os.path.getsize(tflite_path) / 1024  # KB
        print(f"  TensorFlow Lite model size: {model_size:.1f} KB")
        
    except Exception as e:
        print(f"✗ TensorFlow Lite export failed: {e}")
    
    # 5. Save model architecture as JSON
    architecture_path = f"{export_dir}/{model_name}_architecture.json"
    with open(architecture_path, 'w') as f:
        f.write(model.to_json())
    print(f"✓ Saved model architecture: {architecture_path}")
    
    # 6. Save model summary
    summary_path = f"{export_dir}/{model_name}_summary.txt"
    with open(summary_path, 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"✓ Saved model summary: {summary_path}")
    
    return export_dir

def create_inference_class(model_path, target_key='t'):
    """Create a production-ready inference class."""
    
    inference_code = f'''
import tensorflow as tf
import numpy as np
import cv2
from ultralytics import YOLO

class KeypressDetector:
    def __init__(self, model_path="{model_path}", yolo_model_path="best_v2.pt"):
        """Initialize the keypress detector."""
        self.model = tf.keras.models.load_model(model_path)
        self.yolo_model = YOLO(yolo_model_path)
        self.target_key = "{target_key}"
        self.key_bbox = None
        
    def detect_key_region(self, frame):
        """Detect key region in frame using YOLO."""
        results = self.yolo_model(frame)
        
        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    if result.names[int(box.cls)] == self.target_key:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        
                        # Expand bounding box by 20%
                        width = x2 - x1
                        height = y2 - y1
                        expand_x = width * 0.2
                        expand_y = height * 0.2
                        
                        exp_x1 = max(0, int(x1 - expand_x))
                        exp_y1 = max(0, int(y1 - expand_y))
                        exp_x2 = min(frame.shape[1], int(x2 + expand_x))
                        exp_y2 = min(frame.shape[0], int(y2 + expand_y))
                        
                        self.key_bbox = (exp_x1, exp_y1, exp_x2, exp_y2)
                        return self.key_bbox
        
        return None
    
    def preprocess_frame(self, frame, bbox=None):
        """Preprocess frame for model input."""
        if bbox is None:
            bbox = self.key_bbox
        
        if bbox is None:
            return None
        
        # Extract key region
        x1, y1, x2, y2 = bbox
        key_region = frame[y1:y2, x1:x2]
        
        # Resize to model input size
        key_region = cv2.resize(key_region, (64, 64))
        
        # Normalize
        key_region = key_region.astype(np.float32) / 255.0
        
        # Add batch dimension
        key_region = np.expand_dims(key_region, axis=0)
        
        return key_region
    
    def predict(self, frame, bbox=None):
        """Predict keypress from frame."""
        # Preprocess frame
        input_data = self.preprocess_frame(frame, bbox)
        
        if input_data is None:
            return None, None
        
        # Make prediction
        prediction = self.model.predict(input_data, verbose=0)
        
        # Get result
        prob = prediction[0]
        predicted_class = np.argmax(prob)
        confidence = prob[predicted_class]
        
        return predicted_class, confidence
    
    def process_video(self, video_path, output_path=None):
        """Process entire video and return predictions."""
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            raise ValueError(f"Cannot open video: {{video_path}}")
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Setup video writer if output path provided
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        predictions = []
        frame_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # Detect key region on first frame
            if self.key_bbox is None:
                self.detect_key_region(frame)
            
            # Make prediction
            predicted_class, confidence = self.predict(frame)
            
            if predicted_class is not None:
                predictions.append({{
                    'frame': frame_count,
                    'prediction': predicted_class,
                    'confidence': confidence
                }})
                
                # Draw prediction on frame
                label = "PRESSED" if predicted_class == 1 else "NOT PRESSED"
                color = (0, 255, 0) if predicted_class == 1 else (0, 0, 255)
                
                cv2.putText(frame, f"{{label}} ({{confidence:.3f}})", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                
                # Draw bounding box
                if self.key_bbox:
                    x1, y1, x2, y2 = self.key_bbox
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Write frame to output
            if output_path:
                out.write(frame)
        
        # Cleanup
        cap.release()
        if output_path:
            out.release()
        
        return predictions

# Example usage:
# detector = KeypressDetector()
# predictions = detector.process_video("input_video.mp4", "output_video.mp4")
'''
    
    return inference_code

def save_inference_code(code, export_dir):
    """Save inference code to file."""
    code_path = f"{export_dir}/keypress_detector.py"
    with open(code_path, 'w') as f:
        f.write(code)
    print(f"✓ Saved inference code: {code_path}")
    return code_path

def create_deployment_requirements():
    """Create requirements.txt for deployment."""
    requirements = '''
tensorflow>=2.8.0
opencv-python>=4.5.0
ultralytics>=8.0.0
numpy>=1.20.0
'''
    return requirements

def save_deployment_files(export_dir):
    """Save deployment files."""
    
    # Save requirements.txt
    requirements_path = f"{export_dir}/requirements.txt"
    with open(requirements_path, 'w') as f:
        f.write(create_deployment_requirements())
    print(f"✓ Saved requirements: {requirements_path}")
    
    # Save README
    readme_content = f'''
# Keypress Detection Model

This directory contains the trained keypress detection model and inference code.

## Files:
- `keypress_detector.h5`: Trained Keras model
- `keypress_detector_savedmodel/`: TensorFlow SavedModel format
- `keypress_detector.tflite`: TensorFlow Lite model for mobile
- `keypress_detector.py`: Inference class for production use
- `requirements.txt`: Required Python packages

## Usage:
```python
from keypress_detector import KeypressDetector

# Initialize detector
detector = KeypressDetector("keypress_detector.h5")

# Process video
predictions = detector.process_video("input.mp4", "output.mp4")
```

## Installation:
```bash
pip install -r requirements.txt
```

Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
'''
    
    readme_path = f"{export_dir}/README.md"
    with open(readme_path, 'w') as f:
        f.write(readme_content)
    print(f"✓ Saved README: {readme_path}")

# Export the model for production
if 'model' in locals():
    # Export model
    export_dir = export_model(model, "keypress_detector")
    
    # Create inference code
    model_path = f"{export_dir}/keypress_detector.h5"
    inference_code = create_inference_class(model_path)
    save_inference_code(inference_code, export_dir)
    
    # Save deployment files
    save_deployment_files(export_dir)
    
    print(f"\n🎉 Model successfully exported to: {export_dir}")
    print("\nProduction-ready files created:")
    print("- Model files (H5, SavedModel, TensorFlow Lite)")
    print("- Inference code (keypress_detector.py)")
    print("- Requirements and documentation")
    print("\nYour model is ready for deployment!")
    
else:
    print("Model not available for export")
    print("Please train the model first")